# Boilerplate

Package installation, loading, and dataloaders. There's also a simple model defined. You can change it your favourite architecture if you want.

In [3]:
# !pip install tensorboardX

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import time
import matplotlib.pyplot as plt

from torchvision import datasets, transforms
# from tensorboardX import SummaryWriter

use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")
batch_size = 64

np.random.seed(42)
torch.manual_seed(42)


## Dataloaders
train_dataset = datasets.MNIST('mnist_data/', train=True, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))
test_dataset = datasets.MNIST('mnist_data/', train=False, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Simple NN. You can change this if you want. If you change it, mention the architectural details in your report.
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc = nn.Linear(28*28, 50)
        self.fc2 = nn.Linear(50,50)
        self.fc3 = nn.Linear(50,50)
        self.fc4 = nn.Linear(50,10)

    def forward(self, x):
        x = x.view((-1, 28*28))
        x = F.relu(self.fc(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

class Normalize(nn.Module):
    def forward(self, x):
        return (x - 0.1307)/0.3081

# Add the data normalization as a first "layer" to the network
# this allows us to search for adverserial examples to the real image, rather than
# to the normalized image
model = nn.Sequential(Normalize(), Net())

model = model.to(device)
model.train()

Sequential(
  (0): Normalize()
  (1): Net(
    (fc): Linear(in_features=784, out_features=50, bias=True)
    (fc2): Linear(in_features=50, out_features=50, bias=True)
    (fc3): Linear(in_features=50, out_features=50, bias=True)
    (fc4): Linear(in_features=50, out_features=10, bias=True)
  )
)

# Implement Standard Training

In [4]:
def train_model(model, num_epochs):
    # TODO: implement this function that trains a given model on the MNIST dataset.
    # this is a general-purpose function for both standard training and adversarial training.
    # (toggle enable_defense parameter to switch between training schemes)
    model.train()
    optimizer = optim.SGD(model.parameters())

    for epoch in range(num_epochs):


        for data, label in train_loader:
            data = data.to(device)
            label = label.to(device)


            #standard training
            optimizer.zero_grad()
            out = model(data)
            loss = F.cross_entropy(out, label)

            loss.backward()
            optimizer.step()


In [8]:
#Interval analysis
def interval_analysis(model, input, eps):

    lb = input - eps
    ub = input + eps

    lb = torch.clamp(lb, 0, 1)
    ub = torch.clamp(ub, 0, 1)
    
    lb_out = model(lb)
    ub_out = model(ub)

    return lb_out, ub_out

In [5]:
def test_model(model):
    # TODO: implement this function to test the robust accuracy of the given model
    # use pgd_untargeted() within this function

    model.eval()

    correct, total = 0, 0

    for data, label in test_loader:
        data = data.to(device)
        label = label.to(device)

        out = model(data)
        _, predicted = torch.max(out.data, 1)
        
        # print(label.size(0))
        # print(data.size(0))
        # print(out.size(0))
        total += label.size(0)
        # print(predicted)
        correct += (predicted == label).sum().item()

    print("accuracy", 100 * correct / total)
    

In [10]:
def test_robustness(model):

    model.eval()

    for eps in [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]:
        correct, total = 0, 0

        for data, label in test_loader:
            data = data.to(device)
            label = label.to(device)

            out_lb, out_ub = interval_analysis(model, data, eps)
            out_lb, out_ub = out_lb.argmax(dim=1), out_ub.argmax(dim=1) #choose class for each image
            

            total += label.size(0)
            correct += (out_lb == out_ub).sum().item()
        print("eps:", eps)
        print("percent robust", 100 * correct / total)
        print()
    
        

# Study Accuracy, Quality, etc.

Compare the various results and report your observations on the submission.

In [6]:
## train the original model
model = nn.Sequential(Normalize(), Net())
model = model.to(device)
model.train()

train_model(model, 10)
torch.save(model.state_dict(), 'weights.pt')

In [7]:
## basic test
model = nn.Sequential(Normalize(), Net())
model.load_state_dict(torch.load('weights.pt'))

test_model(model)

accuracy 91.35


In [11]:
#robustness test
test_robustness(model)

    

eps: 0.01
percent robust 99.18

eps: 0.02
percent robust 98.49

eps: 0.03
percent robust 97.76

eps: 0.04
percent robust 96.95

eps: 0.05
percent robust 96.19

eps: 0.06
percent robust 95.43

eps: 0.07
percent robust 94.57

eps: 0.08
percent robust 93.8

eps: 0.09
percent robust 92.98

eps: 0.1
percent robust 92.0

